In [1]:
import pickle
import numpy as np
import pandas as pd
import re

seed = 2023

In [2]:
import os
os.chdir("../../")

In [3]:
from nlpsig_networks.scripts.ffn_baseline_functions import (
    ffn_hyperparameter_search,
)

/storage/ttseriotou/nlpsig-networks-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Rumours

In [4]:
%run notebooks/Rumours/load_sbert-embeddings.py

In [5]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


In [6]:
sbert_embeddings.shape

torch.Size([5568, 384])

Baseline: FFN

In [7]:
num_epochs = 100
batch = 64
hidden_dim_sizes = [[64,64], [128,128], [256,256]]
dropout_rates =  [0.2, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [0, 1, 12, 123, 1234]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

We use the ffn_hyperparameter_search function which loops through the different hidden dimensions, dropout rates and learning rates to find the best model for the validation set. We evaluate the model on several seeds and average the performance over the seeds.

In [8]:
ffn_current, best_ffn_current, _, __ = ffn_hyperparameter_search( 
    num_epochs=num_epochs,
    x_data=sbert_embeddings,
    y_data=y_data,
    hidden_dim_sizes=hidden_dim_sizes,
    output_dim=output_dim,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    batch_size=batch,
    #data_split_seed=0,
    split_ids= None, #torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices = (df_rumours[df_rumours['set']=='train'].index, df_rumours[df_rumours['set']=='dev'].index, df_rumours[df_rumours['set']=='test'].index),
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=None,
    verbose=False
)

  0%|          | 0/3 [00:00<?, ?it/s]



100%|██████████| 3/3 [00:46<00:00, 15.58s/it]




 33%|███▎      | 1/3 [01:30<03:01, 90.63s/it]



100%|██████████| 3/3 [00:41<00:00, 13.94s/it]




 67%|██████▋   | 2/3 [02:53<01:25, 85.98s/it]



100%|██████████| 3/3 [00:42<00:00, 14.08s/it]




100%|██████████| 3/3 [04:19<00:00, 86.67s/it]


In [9]:
ffn_current

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,valid_recall_scores,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold,n_splits,batch_size,model_id
0,focal,0.510010,0.490157,"[0.5907643312101912, 0.38954869358669836]",0.502142,"[0.5247524752475248, 0.47953216374269003]",0.501887,"[0.6757741347905283, 0.328]",0.241552,0.622776,...,"[0.7150837988826816, 0.46078431372549017]","(64, 64)",0.2,0.0010,0,2,False,None,64,0
0,focal,0.513823,0.502683,"[0.5771144278606964, 0.4282511210762332]",0.508463,"[0.5296803652968036, 0.4872448979591837]",0.507940,"[0.6338797814207651, 0.382]",0.234741,0.604982,...,"[0.6815642458100558, 0.47058823529411764]","(64, 64)",0.2,0.0010,1,2,False,None,64,0
0,focal,0.508103,0.494675,"[0.5770491803278689, 0.4123006833712984]",0.501713,"[0.5245901639344263, 0.47883597883597884]",0.501583,"[0.6411657559198543, 0.362]",0.238201,0.612100,...,"[0.6815642458100558, 0.49019607843137253]","(64, 64)",0.2,0.0010,12,2,False,None,64,0
0,focal,0.520496,0.505614,"[0.5913891145410235, 0.41983852364475205]",0.514819,"[0.533724340175953, 0.49591280653950953]",0.513512,"[0.663023679417122, 0.364]",0.235927,0.626335,...,"[0.7150837988826816, 0.47058823529411764]","(64, 64)",0.2,0.0010,123,2,False,None,64,0
0,focal,0.510010,0.495982,"[0.5800653594771242, 0.4118993135011441]",0.503605,"[0.5259259259259259, 0.48128342245989303]",0.503315,"[0.6466302367941712, 0.36]",0.230542,0.608541,...,"[0.6927374301675978, 0.46078431372549017]","(64, 64)",0.2,0.0010,1234,2,False,None,64,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,focal,0.497617,0.437866,"[0.6211358734723221, 0.25459688826025456]",0.473923,"[0.5130641330166271, 0.43478260869565216]",0.483443,"[0.7868852459016393, 0.18]",0.272674,0.637011,...,"[0.8156424581005587, 0.3235294117647059]","(256, 256)",0.1,0.0005,0,2,False,None,64,17
0,focal,0.539562,0.535992,"[0.5766871165644171, 0.4952978056426332]",0.537171,"[0.5557432432432432, 0.5185995623632386]",0.536636,"[0.599271402550091, 0.474]",0.234293,0.629893,...,"[0.6871508379888268, 0.5294117647058824]","(256, 256)",0.1,0.0005,1,2,False,None,64,17
0,focal,0.492850,0.475835,"[0.5702746365105009, 0.3813953488372093]",0.483946,"[0.5123367198838897, 0.45555555555555555]",0.485494,"[0.6429872495446266, 0.328]",0.332601,0.619217,...,"[0.7039106145251397, 0.47058823529411764]","(256, 256)",0.1,0.0005,12,2,False,None,64,17
0,focal,0.496663,0.473285,"[0.584251968503937, 0.36231884057971014]",0.485940,"[0.5145631067961165, 0.4573170731707317]",0.487887,"[0.6757741347905283, 0.3]",0.309040,0.619217,...,"[0.7318435754189944, 0.4215686274509804]","(256, 256)",0.1,0.0005,123,2,False,None,64,17


In [10]:
best_ffn_current

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,valid_recall,valid_recall_scores,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold,n_splits,batch_size
0,focal,0.497617,0.437866,"[0.6211358734723221, 0.25459688826025456]",0.473923,"[0.5130641330166271, 0.43478260869565216]",0.483443,"[0.7868852459016393, 0.18]",0.272674,0.637011,...,0.569586,"[0.8156424581005587, 0.3235294117647059]","(256, 256)",0.1,0.0005,0,2,False,None,64
0,focal,0.539562,0.535992,"[0.5766871165644171, 0.4952978056426332]",0.537171,"[0.5557432432432432, 0.5185995623632386]",0.536636,"[0.599271402550091, 0.474]",0.234293,0.629893,...,0.608281,"[0.6871508379888268, 0.5294117647058824]","(256, 256)",0.1,0.0005,1,2,False,None,64
0,focal,0.492850,0.475835,"[0.5702746365105009, 0.3813953488372093]",0.483946,"[0.5123367198838897, 0.45555555555555555]",0.485494,"[0.6429872495446266, 0.328]",0.332601,0.619217,...,0.587249,"[0.7039106145251397, 0.47058823529411764]","(256, 256)",0.1,0.0005,12,2,False,None,64
0,focal,0.496663,0.473285,"[0.584251968503937, 0.36231884057971014]",0.485940,"[0.5145631067961165, 0.4573170731707317]",0.487887,"[0.6757741347905283, 0.3]",0.309040,0.619217,...,0.576706,"[0.7318435754189944, 0.4215686274509804]","(256, 256)",0.1,0.0005,123,2,False,None,64
0,focal,0.513823,0.496789,"[0.5893719806763285, 0.40420560747663553]",0.507047,"[0.5281385281385281, 0.4859550561797753]",0.506333,"[0.6666666666666666, 0.346]",0.238873,0.626335,...,0.594945,"[0.7094972067039106, 0.4803921568627451]","(256, 256)",0.1,0.0005,1234,2,False,None,64


In [11]:
best_ffn_current["f1"].mean()

0.4839536066523949

In [12]:
best_ffn_current["precision"].mean()

0.49760555870433587

In [13]:
best_ffn_current["recall"].mean()

0.49995846994535514

In [14]:
np.stack(best_ffn_current["f1_scores"]).mean(axis=0)

array([0.58834432, 0.3795629 ])

In [18]:
np.stack(best_ffn_current["precision_scores"]).mean(axis=0)

array([0.52476915, 0.47044197])

In [19]:
np.stack(best_ffn_current["recall_scores"]).mean(axis=0)

array([0.67431694, 0.3256    ])